List of active contributors in this assignment
1. 2020sc04042@wilp.bits-pilani.ac.in
2. 2020sc04050@wilp.bits-pilani.ac.in
3. 2020sc04198@wilp.bits-pilani.ac.in
4. 2020sc04368@wilp.bits-pilani.ac.in

In [ ]:
from contextlib import suppress

variables = {
             "Monday": ["Hour1", "Hour2", "Hour3", "Hour4", "Hour5"],
             "Tuesday" : ["Hour1", "Hour2", "Hour3", "Hour4", "Hour5"],
             "Wednesday" : ["Hour1", "Hour2", "Hour3", "Hour4", "Hour5"],
             "Thursday" : ["Hour1", "Hour2", "Hour3", "Hour4", "Hour5"],
             "Friday" : ["Hour1", "Hour2", "Hour3", "Hour4", "Hour5"]
            }

Domains = ["AssoProf-1", "AssoProf-2", "AssisProf-1", "AssisProf-2", "AssisProf-3", "EngProf", "MathsProf", "HOD"]

ProfessorConstraints = { "Hour1" :["EngProf", "MathsProf", "AssoProf-1", "AssoProf-2", "AssisProf-1", "AssisProf-2", "AssisProf-3" ], 
                        "Hour2" : ["EngProf", "MathsProf", "AssisProf-1", "AssisProf-2", "AssisProf-3","HOD"], 
                        "Hour3" : ["EngProf","AssoProf-1", "AssoProf-2", "MathsProf", "AssisProf-1", "AssisProf-2", "AssisProf-3", "HOD"],
                        "Hour4" : ["EngProf","AssoProf-1", "AssoProf-2", "MathsProf", "AssisProf-1", "AssisProf-2", "AssisProf-3", "HOD"], 
                        "Hour5" : ["EngProf","AssoProf-1", "AssoProf-2", "MathsProf", "AssisProf-1", "AssisProf-2", "AssisProf-3", "HOD"] }

HourConstraints = {
        "Hour1" : 60,
        "Hour2" : 60,
        "Hour3" : 60,
        "Hour4" : 50,
        "Hour5" : 50
}
 

StaffAailability = {
    "HOD": 660,
    "EngProf": 720,
    "MathsProf": 720,
    "AssisProf-1": 720,
    "AssisProf-2": 720,
    "AssisProf-3": 720,
    "AssoProf-1": 720,
    "AssoProf-2": 720
}

LabConstraints = { "Monday": {"Hour1":"Year1L1", "Hour2":"Year2L2", "Hour3":"Year3L3", "Hour4":"Year4L4", "Hour5":""},
                  "Tuesday" : {"Hour1":"Year2L1", "Hour2":"Year3L2", "Hour3":"Year4L3", "Hour4":"Year3L5", "Hour5":""},
                  "Wednesday" : {"Hour1":"Year3L1", "Hour2":"Year4L2", "Hour3":"Year1L4", "Hour4":"Year3L6", "Hour5":""},
                  "Thursday" : {"Hour1":"Year4L1", "Hour2":"Year1L3", "Hour3":"Year2L4", "Hour4":"Year4L5", "Hour5":""},
                  "Friday" : {"Hour1":"Year1L2", "Hour2":"Year2L3", "Hour3":"Year3L4", "Hour4":"Year4L6", "Hour5":""} }
         




Year1assignments = { }
Year2assignments = { }
Year3assignments = { }
Yaer4assignments = { }
assignments = { }
ProfessorTimings = { }
ProfessorAssignment = { }
arcs = []

In [ ]:
#fill queue with initial arcs
def fillQueue():

    for var in variables:
        values = variables[var]
        for i in values:
            for arc in ProfessorConstraints[i]:
                arcs.append([var, i, arc])

In [ ]:
def rev(Xi, Xj, Xk, Year):
    revised = False
    Slots = variables[Xi];
    Professors = ProfessorConstraints[Xj];
    if (Year != "Year1"):
        if "EngProf" in Professors: Professors.remove("EngProf")
        if "MathsProf" in Professors: Professors.remove("MathsProf")
    AllocatedTiming = 0
    assignmentsList = Year + "assignments"
    # for each value in the domain
    for x in Slots:
        Consistent = False
        timeRequired = HourConstraints[x];
        #print("Checking for LAB slot " , Xi , " is " , x , " & LabConstraints[Xi][x] =====> ", LabConstraints[Xi][x])

        with suppress (Exception):
            if (x in LabConstraints[Xi] and str(LabConstraints[Xi][x]).index(Year) > -1):
                #assignments[Xi] = [[x, "Lab"]]
                continue;

        # values = assignments[Xi]
        # if any(x in sublist for sublist in values):
        #   break
        for professor in Professors:
            if (professor in ProfessorAssignment and
                ((professor == "EngProf" and len(ProfessorAssignment [professor]) == 4) or
                (professor == "MathsProf" and len(ProfessorAssignment [professor]) == 4))) :
                #move on to other professor as we have already alloted enough english / maths classes..!
                continue;

            if Xi not in assignments and Consistent:
                continue
            if Xi in assignments:
                values = assignments[Xi];
                res = any(professor in sublist for sublist in values)
                if any(x in sublist for sublist in values):
                    continue
                elif any(professor in sublist for sublist in values):
                    continue

                else:
                    TotalTiming = StaffAailability[professor]
                    if professor in ProfessorTimings:
                        AllocatedTiming = ProfessorTimings[professor]
                    if (AllocatedTiming + timeRequired < TotalTiming):
                        profBlocked = False
                        if professor in ProfessorAssignment:
                            for allocatedSlots in ProfessorAssignment[professor]:
                                # Check if the professor ia already allocated for this time & set the ..!
                                if (Xi == str(allocatedSlots).split('-')[0] and x == str(allocatedSlots).split('-')[1]):
                                    profBlocked = True
                                    break
                        if not profBlocked :
                            assignments[Xi].append([x, professor])
                            ProfessorTimings[professor] = AllocatedTiming + timeRequired
                            revised = True
                        else : continue
                    else: continue

            if not revised:
                TotalTiming = StaffAailability[professor]
                if professor in ProfessorTimings:
                    AllocatedTiming = ProfessorTimings[professor]
                if (AllocatedTiming + timeRequired < TotalTiming):
                    if Xi in Year1assignments.keys():
                        if [x, professor] not in Year1assignments[Xi]:
                            ProfessorTimings[professor] = timeRequired + AllocatedTiming
                            assignments[Xi] = [[x, professor]]
                    else:
                        profBlocked = False
                        if professor in ProfessorAssignment:
                            for allocatedSlots in ProfessorAssignment[professor]:
                                # Check if the professor ia already allocated for this time & set the ..!
                                if (Xi == str(allocatedSlots).split('-')[0] and x == str(allocatedSlots).split('-')[1]):
                                    profBlocked = True
                                    break

                        if not profBlocked :
                            assignments[Xi] = [[x, professor]]
                            ProfessorTimings[professor] = AllocatedTiming + timeRequired
                            revised = True
                        else : continue
                else:
                    continue
            if not professor in ProfessorAssignment:
                profAssign = []
                ProfessorAssignment[professor] = profAssign
            ProfessorAssignment[professor].append(Xi + '-' + x)
    return assignments

In [ ]:
def AC3(Year):
    # ClassYears = ["Year1", "Year2", "Year3", "Year4"]
    # fill queue with initial arcs
    fillQueue()
    # check if arcs is empty
    while arcs:
        arcVars = arcs.pop(0)
        assignments = rev(arcVars[0], arcVars[1], arcVars[2], Year)  # T or F
    return assignments

In [ ]:
yearAssignment = {}

ClassYears = ["Year1", "Year2", "Year3", "Year4"]
for i in ClassYears:
    #print(i)
    assignments ={}
    yearAssignment[i] = AC3(i)
    #print("Professor Allocation Status for ", i ," is  ===> ", ProfessorAssignment)
    #print("Assignment ==> ", yearAssignment)
#print("Professor Allocation Status for ====>" , ProfessorAssignment)
print("Professor Capacity")
print(ProfessorTimings)
print('\n')
print('Year wise allocation')
print(yearAssignment)

Professor Capacity
{'EngProf': 240, 'MathsProf': 240, 'AssoProf-1': 690, 'AssoProf-2': 710, 'AssisProf-1': 710, 'AssisProf-2': 700, 'AssisProf-3': 700, 'HOD': 610}


Year wise allocation
{'Year1': {'Monday': [['Hour2', 'EngProf'], ['Hour3', 'MathsProf'], ['Hour4', 'AssoProf-1'], ['Hour5', 'AssoProf-2']], 'Tuesday': [['Hour1', 'EngProf'], ['Hour2', 'MathsProf'], ['Hour3', 'AssoProf-1'], ['Hour4', 'AssoProf-2'], ['Hour5', 'AssisProf-1']], 'Wednesday': [['Hour1', 'EngProf'], ['Hour2', 'MathsProf'], ['Hour4', 'AssoProf-1'], ['Hour5', 'AssoProf-2']], 'Thursday': [['Hour1', 'EngProf'], ['Hour3', 'MathsProf'], ['Hour4', 'AssoProf-1'], ['Hour5', 'AssoProf-2']], 'Friday': [['Hour2', 'AssoProf-1'], ['Hour3', 'AssoProf-2'], ['Hour4', 'AssisProf-1'], ['Hour5', 'AssisProf-2']]}, 'Year2': {'Monday': [['Hour1', 'AssoProf-1'], ['Hour3', 'AssoProf-2'], ['Hour4', 'AssisProf-1'], ['Hour5', 'AssisProf-2']], 'Tuesday': [['Hour2', 'AssoProf-1'], ['Hour3', 'AssoProf-2'], ['Hour4', 'AssisProf-1'], ['Hour5', '